In [1]:
!pip install azure-mgmt-resource

StatementMeta(, 0b8ecc26-831b-482c-9360-7bdeb27858ca, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 44.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.0.0 requires sentencepiece, which is not installed.
sentence-transformers 2.0.0 requires torchvision, which is not installed.
dash 2.14.0 requires Flask<2.3.0,>=1.0.4, but you have flask 3.0.0 which is incompatible.
dash 2.14.0 requires Werkzeug<2.3.0, but you have werkzeug 3.0.1 which is incompatible.
tensorflow 2.12.1 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [2]:
!pip install azure-identity

StatementMeta(, 0b8ecc26-831b-482c-9360-7bdeb27858ca, 4, Finished, Available, Finished)

In [10]:
import os, json,datetime
from azure.mgmt.resource import ResourceManagementClient
from azure.identity import ClientSecretCredential
from azure.mgmt.resource.resources.models import DeploymentMode
import pandas as pd

#### Set these variables
# Name of the Key Vault
nameOfKeyVault = 'aquatickeyvault' # Name of the Key Vault
# Names of the secrets saved in Key Vault
tenantId_SecretName = 'bcdrtenantid' # Name for Tenant ID
clientId_SecretName = 'bcdrserviceprincipalid'   # Name for Client ID of Service Principal
clientSecret_SecretName = 'bcdrserviceprincipalsecret' # Name for Client Secret of Service Principal
# Specify the target subscription ID
subscription_id = 'ac7ff781-1f82-40ff-ba6c-06fc32d82862'
# Specify the target resource group
rg_name = 'rgfabmig'
# Specify the location
location = 'UK South'

# Specify the Fabric capacity name
cap_name = 'fabmig1'

today = datetime.datetime.now().strftime("%m-%d-%Y")
#deployment_name = f"CapacityName{today}"
deployment_name = cap_name + "deployment"

# Specify the SKU
fabric_sku = 'F2'

keyvault = f'https://{nameOfKeyVault}.vault.azure.net/'
tenant_id = mssparkutils.credentials.getSecret(keyvault,tenantId_SecretName)
client_id = mssparkutils.credentials.getSecret(keyvault,clientId_SecretName)
client_secret = mssparkutils.credentials.getSecret(keyvault,clientSecret_SecretName)
credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret,
)

resource_client = ResourceManagementClient(credential, subscription_id)

print('Creating resource group ' + rg_name + ' in region ' + location + '...')
# First create a resource group if it doesn't exist
rg_result = resource_client.resource_groups.create_or_update(
    rg_name,
    {
        "location": location
    }
)

print(f"Provisioned resource group with ID: {rg_result.id}")


StatementMeta(, 0b8ecc26-831b-482c-9360-7bdeb27858ca, 12, Finished, Available, Finished)

Creating resource group rgfabmig in region UK South...
Provisioned resource group with ID: /subscriptions/ac7ff781-1f82-40ff-ba6c-06fc32d82862/resourceGroups/rgfabmig


In [31]:

template = '''
{
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentTemplate.json#",
    "contentVersion": "1.0.0.1",
    "parameters": {
        "name": {
            "type": "string"
        },
        "location": {
            "type": "string"
        },
        "sku": {
            "type": "string",
            "allowedValues": [
                "F2",
                "F4",
                "F8",
                "F16",
                "F32",
                "F64",
                "F128",
                "F256",
                "F512",
                "F1024",
                "F2048"
            ]
        },
        "admin": {
            "type": "string"
        },
        "tagValues": {
            "type": "object",
            "defaultValue": {}
        }
    },
    "variables": {},
    "resources": [
        {
            "apiVersion": "2022-07-01-preview",
            "name": "[parameters('name')]",
            "location": "[parameters('location')]",
            "sku": {
                "name": "[parameters('sku')]",
                "tier": "Fabric"
            },
            "properties": {
                "administration": {
                    "members": [
                        "[parameters('admin')]"
                    ]
                }
            },
            "type": "Microsoft.Fabric/capacities",
            "tags": "[parameters('tagValues')]"
        }
    ],
    "outputs": {}
}
'''
 
parameters = '''
{
        "name": {
            "value": "''' + cap_name + '''"
        },
        "location": {
            "value": "''' + location + '''"
        },
        "sku": {
            "value": "''' + fabric_sku + '''"
        },
        "admin": {
            "value": "admin@MngEnv572680.onmicrosoft.com"
        },
        "tagValues": {
            "value": {}
        }
    }
'''

# Deployment properties
deployment_properties = {"properties": {"template": json.loads(template), "parameters": json.loads(parameters),"mode": DeploymentMode.incremental}}

    #################################### 
    # ARM template what-if
    ###################################

    print('conducting what-if of Fabric capacity deployment '+deployment_name + ' in region ' +location+'...')
    what_if = resource_client.deployments.begin_what_if(rg_name, deployment_name, deployment_properties)
    what_if.wait()
    #for i in deployment_async_operation.result().changes:
    #    print(i.change_type + ' resource '+ i.resource_id + ' in location '+ i.after['location'] + ' with capacity admins '+ str(i.after['properties']['administration']['members']))
    df = pd.concat([
    pd.DataFrame({
        'Change': [i.change_type for i in what_if.result().changes],
        'Type': [i.after['type'] for i in what_if.result().changes],
        'Name':[i.after['name'] for i in what_if.result().changes],
        'Location':[i.after['location'] for i in what_if.result().changes],
        'Sku': [i.after['sku'] for i in what_if.result().changes],
        'Group': [i.resource_id.split('/')[4] for i in what_if.result().changes], 
        'Resource': [i.resource_id for i in what_if.result().changes] 
    })], ignore_index=True)   
    display(df)
    print('deployment what-if output complete')

    #################################### 
    # Validate ARM template deployment
    ###################################
    print('validating deployment of Fabric capacity '+deployment_name + ' in region ' +location+'...')
    validation_async_operation = resource_client.deployments.begin_validate(rg_name, deployment_name, deployment_properties)
    validation_async_operation.wait()
    print('deployment validation '+str(validation_async_operation.status()).lower())

    if validation_async_operation.status()=='Succeeded' and validation_async_operation.result().error is None:

        #################################### 
        # ARM template deployment
        ###################################

        print('deploying Fabric capacity '+deployment_name + ' in region ' +location+'...')
        deployment_async_operation = resource_client.deployments.begin_create_or_update(rg_name, deployment_name, deployment_properties)
        deployment_async_operation.wait()
        print('deployment complete')
    else: 
        print('Validation of capacity deployment '+deployment_name+' failed due to: '+validation_async_operation.result().error +'. Please resolve and retry.')    


StatementMeta(, 0b8ecc26-831b-482c-9360-7bdeb27858ca, 33, Finished, Available, Finished)

conducting what-if of Fabric capacity deployment fabmig1deployment in region UK South...


SynapseWidget(Synapse.DataFrame, 9be965c1-fef3-418b-ad05-41809a8a38bf)

deployment what-if output complete


In [46]:
#################################### 
# rollback: delete capacities
###################################
for i in deployment_async_operation.result().properties.output_resources:
    print('deleting resource '+i.id)
    delete_async_operation = resource_client.resources.begin_delete_by_id(i.id,'2023-11-01')
    delete_async_operation.wait()
    if delete_async_operation.status()=='Succeeded':
        print('deployment deleted')
    else:
        print('failed to delete '+ deployment_name +'. Error '+delete_async_operation.result().error +'. Please delete manually if necessary.')


StatementMeta(, 0b8ecc26-831b-482c-9360-7bdeb27858ca, 48, Finished, Available, Finished)

deleting resource /subscriptions/ac7ff781-1f82-40ff-ba6c-06fc32d82862/resourceGroups/rgfabmig/providers/Microsoft.Fabric/capacities/fabmig1
deployment deleted
